<a href="https://colab.research.google.com/github/pinzar14/Procesamiento_digital_de_seniales/blob/main/3PD_Seniales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###3er tarea Procesamiento digital de señales

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, IntSlider
from matplotlib.gridspec import GridSpec

In [9]:
#------------------------------------------------------------- Parámetros fijos-----------------------------------------------------------------------------------------
fs = 2000  # Frecuencia de muestreo
frecs = np.arange(10, 510, 10)  # Frecuencias (sen,cos)

def bandstop_filter(signal_fft, freqs_fft, f_low, f_high):
    """Aplica un filtro de banda eliminada al espectro de la señal."""
    filtered_fft = signal_fft.copy()
    band = np.where((np.abs(freqs_fft) >= f_low) & (np.abs(freqs_fft) <= f_high))
    filtered_fft[band] = 0
    return filtered_fft

def equalizer(signal_fft, freqs_fft, f_low, f_high, gain_db):
    """Aplica una ganancia a una banda de frecuencias específica."""
    gain = 10**(gain_db / 20)
    equalized_fft = signal_fft.copy()
    band = np.where((np.abs(freqs_fft) >= f_low) & (np.abs(freqs_fft) <= f_high))
    equalized_fft[band] *= gain
    return equalized_fft

def calculate_thd(signal, fundamental_freq, sampling_rate):
    """Calcula la Distorsión Armónica Total (THD) de una señal."""
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / sampling_rate)
    fundamental_index = np.argmin(np.abs(xf - fundamental_freq))
    harmonic_indices = np.where(np.abs(xf) > fundamental_freq * 1.1)  # Excluye la fundamental y frecuencias cercanas
    fundamental_power = np.abs(yf[fundamental_index])**2
    harmonic_power = np.sum(np.abs(yf[harmonic_indices])**2)
    thd = np.sqrt(harmonic_power / fundamental_power)
    return thd

def analyze_signal(num_samples, fb_low, fb_high, feq_low, feq_high, gain_db):
    """Genera la señal, aplica filtros y ecualización, y muestra los resultados."""
    t = np.arange(0, num_samples) / fs  # Tiempo ajustado
    x = np.sum([np.cos(2 * np.pi * f * t) for f in frecs], axis=0)
    X = fft(x)
    freqs_fft = fftfreq(len(x), 1 / fs)

    # Filtro bandstop
    X_bandstop = bandstop_filter(X, freqs_fft, fb_low, fb_high)
    x_bandstop = ifft(X_bandstop).real

    # Ecualizador
    X_eq = equalizer(X_bandstop, freqs_fft, feq_low, feq_high, gain_db)
    x_eq = ifft(X_eq).real

    # Cálculo de THD (usando la frecuencia fundamental más baja como ejemplo)
    thd_original = calculate_thd(x, frecs[0], fs)
    thd_bandstop = calculate_thd(x_bandstop, frecs[0], fs)
    thd_equalized = calculate_thd(x_eq, frecs[0], fs)

    # Gráficas con GridSpec para mayor control
    fig = plt.figure(figsize=(16, 10))
    gs = GridSpec(3, 2, width_ratios=[2, 1], height_ratios=[1, 1, 1])

    # Señal original
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(t * 1000, x, color='skyblue')  # Tiempo en ms
    ax1.set_title(f"Señal Original (THD: {thd_original:.2f})")
    ax1.set_xlabel("Tiempo [ms]")
    ax1.set_ylabel("Amplitud")
    ax1.grid(True, linestyle='--', alpha=0.6)

    ax2 = fig.add_subplot(gs[0, 1])
    ax2.stem(freqs_fft[:len(freqs_fft)//2], np.abs(X[:len(X)//2]), basefmt=" ", linefmt='cadetblue', markerfmt='o')
    ax2.set_title("Espectro Original")
    ax2.set_xlabel("Frecuencia [Hz]")
    ax2.set_ylabel("Magnitud")
    ax2.grid(True, linestyle='--', alpha=0.6)

    # Señal después del filtro bandstop
    ax3 = fig.add_subplot(gs[1, 0])
    ax3.plot(t * 1000, x_bandstop, color='salmon') # Tiempo en ms
    ax3.set_title(f"Señal Bandstop ({fb_low}-{fb_high} Hz, THD: {thd_bandstop:.2f})")
    ax3.set_xlabel("Tiempo [ms]")
    ax3.set_ylabel("Amplitud")
    ax3.grid(True, linestyle='--', alpha=0.6)

    ax4 = fig.add_subplot(gs[1, 1])
    ax4.stem(freqs_fft[:len(freqs_fft)//2], np.abs(X_bandstop[:len(X)//2]), basefmt=" ", linefmt='coral', markerfmt='o')
    ax4.set_title("Espectro Bandstop")
    ax4.set_xlabel("Frecuencia [Hz]")
    ax4.set_ylabel("Magnitud")
    ax4.grid(True, linestyle='--', alpha=0.6)

    # Señal después del ecualizador
    ax5 = fig.add_subplot(gs[2, 0])
    ax5.plot(t * 1000, x_eq, color='mediumpurple') # Tiempo en ms
    ax5.set_title(f"Señal Ecualizada ({feq_low}-{feq_high} Hz, {gain_db} dB, THD: {thd_equalized:.2f})")
    ax5.set_xlabel("Tiempo [ms]")
    ax5.set_ylabel("Amplitud")
    ax5.grid(True, linestyle='--', alpha=0.6)

    ax6 = fig.add_subplot(gs[2, 1])
    ax6.stem(freqs_fft[:len(freqs_fft)//2], np.abs(X_eq[:len(X)//2]), basefmt=" ", linefmt='darkorchid', markerfmt='o')
    ax6.set_title("Espectro Ecualizado")
    ax6.set_xlabel("Frecuencia [Hz]")
    ax6.set_ylabel("Magnitud")
    ax6.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

# Widgets interactivos
interact(
    analyze_signal,
    num_samples=IntSlider(min=500, max=20000, step=500, value=2000, description='N muestras'),
    fb_low=FloatSlider(min=0, max=1000, step=10, value=100, description='Stop f_low (Hz)'),
    fb_high=FloatSlider(min=0, max=1000, step=10, value=150, description='Stop f_high (Hz)'),
    feq_low=FloatSlider(min=0, max=1000, step=10, value=300, description='EQ f_low (Hz)'),
    feq_high=FloatSlider(min=0, max=1000, step=10, value=350, description='EQ f_high (Hz)'),
    gain_db=FloatSlider(min=-20, max=20, step=1, value=0, description='Ganancia EQ (dB)')
);

interactive(children=(IntSlider(value=2000, description='N muestras', max=20000, min=500, step=500), FloatSlid…